In [2]:
import pandas as pd 

In [3]:
data = pd.read_excel("dataset.xlsx", sheet_name="Run 1_ PV_battery_model")

In [4]:
data.head()

,time,ILabc(1),ILabc(2),ILabc(3),Vgrid(1),Vgrid(2),Vgrid(3),Igrid(1),Igrid(2),Igrid(3),SOC,Iev,Ipv,Vev,Vpv,Snfc,Snet
0,0.00000,0.048660,-1.590622,1.541962,-0.000020,-267.261993,267.262014,-0.048660,1.596234,-1.547575,60.0,0.0,0.0,0.0,0.0,0.0,4.849652e-07
1,0.00001,0.052998,-1.559174,1.506177,1.408322,-269.639370,268.231048,0.657480,0.859739,-1.517219,60.0,0.0,0.0,0.0,0.0,0.0,1.699499e-08
2,0.00002,0.052162,-1.509380,1.457218,1.946813,-269.255735,267.308922,0.653073,1.532406,-2.185479,60.0,0.0,0.0,0.0,0.0,0.0,-2.321159e-06
3,0.00003,0.053851,-1.475135,1.421284,2.916190,-269.720708,266.804517,0.881223,1.984728,-2.865950,60.0,0.0,0.0,0.0,0.0,0.0,-1.396962e-05
4,0.00004,0.055353,-1.447272,1.391920,3.894597,-270.197179,266.302582,0.867524,2.920913,-3.788437,60.0,0.0,0.0,0.0,0.0,0.0,-7.116881e-05


In [5]:
data= data.drop(["time" , "Snfc"] , axis=1)

In [6]:
data.head()

,ILabc(1),ILabc(2),ILabc(3),Vgrid(1),Vgrid(2),Vgrid(3),Igrid(1),Igrid(2),Igrid(3),SOC,Iev,Ipv,Vev,Vpv,Snet
0,0.048660,-1.590622,1.541962,-0.000020,-267.261993,267.262014,-0.048660,1.596234,-1.547575,60.0,0.0,0.0,0.0,0.0,4.849652e-07
1,0.052998,-1.559174,1.506177,1.408322,-269.639370,268.231048,0.657480,0.859739,-1.517219,60.0,0.0,0.0,0.0,0.0,1.699499e-08
2,0.052162,-1.509380,1.457218,1.946813,-269.255735,267.308922,0.653073,1.532406,-2.185479,60.0,0.0,0.0,0.0,0.0,-2.321159e-06
3,0.053851,-1.475135,1.421284,2.916190,-269.720708,266.804517,0.881223,1.984728,-2.865950,60.0,0.0,0.0,0.0,0.0,-1.396962e-05
4,0.055353,-1.447272,1.391920,3.894597,-270.197179,266.302582,0.867524,2.920913,-3.788437,60.0,0.0,0.0,0.0,0.0,-7.116881e-05


In [7]:
datax = data.drop(columns=['Snet'])

In [8]:
features = datax.columns

In [9]:
from sklearn import preprocessing
scaler = preprocessing.StandardScaler()
scaler.fit(datax)
dfx1=scaler.transform(datax)
dfx1 = pd.DataFrame(dfx1)

In [10]:
dfx1.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,0.001936,-0.192224,0.190654,0.000001,-1.218123,1.218122,0.007509,0.015704,-0.023221,-2.089564,-0.129086,-3.860572,-8.886841,-8.756152
1,0.002471,-0.188348,0.186234,0.006420,-1.228959,1.222539,0.018867,0.003853,-0.022732,-2.089564,-0.129086,-3.860572,-8.886841,-8.756152
2,0.002368,-0.182210,0.180188,0.008874,-1.227210,1.218336,0.018796,0.014677,-0.033487,-2.089564,-0.129086,-3.860572,-8.886841,-8.756152
3,0.002576,-0.177989,0.175750,0.013292,-1.229329,1.216037,0.022466,0.021956,-0.044439,-2.089564,-0.129086,-3.860572,-8.886841,-8.756152
4,0.002761,-0.174554,0.172123,0.017752,-1.231501,1.213749,0.022245,0.037021,-0.059285,-2.089564,-0.129086,-3.860572,-8.886841,-8.756152


In [11]:
dfy1 = data['Snet']

In [12]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dfx1,dfy1,test_size=0.2, random_state=42) 

In [13]:
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_regression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from joblib import Parallel, delayed  # For parallel processing

# Particle class for PSO
class Particle:
    def __init__(self, dim, bounds):
        self.position = np.array([random.uniform(bounds[d][0], bounds[d][1]) for d in range(dim)])
        self.velocity = np.zeros(dim)
        self.best_position = np.copy(self.position)
        self.best_score = float('inf')

def pso_optimize(func, dim, bounds, n_particles=7, max_iter=20, w=0.5, c1=1.5, c2=1.5, early_stop=5):
    particles = [Particle(dim, bounds) for _ in range(n_particles)]
    global_best = min(particles, key=lambda p: func(p.position))

    best_loss = func(global_best.position)
    no_improve_count = 0

    for _ in range(max_iter):
        # Parallel evaluation of particle fitness
        scores = Parallel(n_jobs=-1)(delayed(func)(p.position) for p in particles)

        for i, particle in enumerate(particles):
            score = scores[i]

            if score < particle.best_score:
                particle.best_score = score
                particle.best_position = np.copy(particle.position)

            if score < best_loss:
                global_best = particle
                best_loss = score
                no_improve_count = 0
            else:
                no_improve_count += 1

            # Early stopping
            if no_improve_count >= early_stop:
                return global_best.position, best_loss

            # Update velocity and position
            inertia = w * particle.velocity
            cognitive = c1 * random.random() * (particle.best_position - particle.position)
            social = c2 * random.random() * (global_best.position - particle.position)
            particle.velocity = inertia + cognitive + social

            # Update position and ensure it stays within bounds
            particle.position += particle.velocity
            particle.position = np.clip(particle.position, [b[0] for b in bounds], [b[1] for b in bounds])

    return global_best.position, best_loss

# Function to optimize regression model using PSO
def regression_pso(model_class, param_bounds, X_train, X_test, y_train, y_test, n_particles=7, max_iter=12):
    def model_fitness(params):
        # Convert parameters to correct types (int for discrete values)
        params = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), params)}
        model = model_class(**params)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        return mean_squared_error(y_test, y_pred)

    bounds = list(param_bounds.values())
    best_params, best_loss = pso_optimize(model_fitness, len(bounds), bounds, n_particles, max_iter)

    # Convert best parameters to correct types
    best_params_dict = {key: int(val) if isinstance(param_bounds[key][0], int) else val for key, val in zip(param_bounds.keys(), best_params)}
    return best_params_dict, best_loss

# Generate sample regression data
# X, Y = make_regression(n_samples=1000, n_features=10, noise=0.1, random_state=42)


In [ ]:

if __name__ == "__main__":
    models = {
        "GradientBoosting": (GradientBoostingRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "XGBRegressor": (XGBRegressor, {"n_estimators": (50, 100), "learning_rate": (0.05, 0.2)}),
        "RandomForest": (RandomForestRegressor, {"n_estimators": (50, 100), "max_depth": (1, 20)}),
        
        "DecisionTree": (DecisionTreeRegressor, {"max_depth": (1, 20), "min_samples_split": (2, 10)}),
    }

    for name, (model_class, param_bounds) in models.items():
        print(f"Optimizing {name} using PSO...")
        best_params, best_loss = regression_pso(model_class, param_bounds, X_train, X_test, y_train , y_test)
        print(f"Best Parameters: {best_params}, Best Loss: {best_loss}\n")


Optimizing GradientBoosting using PSO...
Best Parameters: {'n_estimators': 84, 'learning_rate': 0.17822224171704676}, Best Loss: 1.908906645102929

Optimizing XGBRegressor using PSO...
Best Parameters: {'n_estimators': 82, 'learning_rate': 0.1825958416173385}, Best Loss: 0.12555080601850604

Optimizing RandomForest using PSO...
Best Parameters: {'n_estimators': 80, 'max_depth': 19}, Best Loss: 0.041975028178210874

Optimizing DecisionTree using PSO...
Best Parameters: {'max_depth': 17, 'min_samples_split': 7}, Best Loss: 0.06746606134672448



In [ ]:
from sklearn.svm import SVR
if __name__ == "__main__":
    models = {
        "SVR": (SVR, {"C": (1e-3, 1e3), "epsilon":(1e-4, 1.0) , "gamma":(1e-4, 1e1)}),
        
        }

    for name, (model_class, param_bounds) in models.items():
        print(f"Optimizing {name} using PSO...")
        best_params, best_loss = regression_pso(model_class, param_bounds, X_train, X_test, y_train , y_test)
        print(f"Best Parameters: {best_params}, Best Loss: {best_loss}\n")

Optimizing SVR using PSO...
